In [1]:
import torch.nn as nn
import torch
import math

batch_size = 1
image_size = 50
dim_in = 3
emb_dim = 24
c_group = 1
kernel_size = 3
stride = 2
c_group = 1
gamma = 0.5
beta = 0.2

running_mean = torch.zeros(emb_dim)
running_var = torch.zeros(emb_dim)
for i in range(emb_dim):
    running_mean[i] = 8
    running_var[i] = 21.5

image = torch.zeros(batch_size, dim_in, image_size, image_size)
for b in range(batch_size):
    for c in range(dim_in):
        for i in range(image_size):
            for j in range(image_size):
                image[b, c, i, j] = (c+i+j)%256

print('this is the image')
print(image[0, 0, :, :])

kernel = torch.zeros(emb_dim, dim_in, kernel_size, kernel_size)
kernel[:, :, :, :] = 0.1
bias = torch.zeros((emb_dim,))
for c in range(emb_dim):
    bias[c] = c + 0.01

c_group = math.gcd(dim_in, emb_dim) if c_group == -1 else c_group
padding = math.ceil(((kernel_size - 1) * 1 + 1 - stride) / 2)
print('padding: ', padding)
conv = nn.Conv2d(dim_in, emb_dim, kernel_size, stride, padding, dilation=1, groups=c_group, bias=True)
conv.weight.data = kernel
conv.bias.data = bias
image = conv(image)
print('after conv')
print(image[0, 0, :, :])

norm = nn.BatchNorm2d(emb_dim, momentum=0)
norm.running_mean = running_mean
norm.running_var = running_var
norm.weight.data.fill_(gamma)
norm.bias.data.fill_(beta)
norm.eval()
result = norm(image)

print(result.shape)
print(result[0, 0, :, :])

this is the image
tensor([[ 0.,  1.,  2.,  ..., 47., 48., 49.],
        [ 1.,  2.,  3.,  ..., 48., 49., 50.],
        [ 2.,  3.,  4.,  ..., 49., 50., 51.],
        ...,
        [47., 48., 49.,  ..., 94., 95., 96.],
        [48., 49., 50.,  ..., 95., 96., 97.],
        [49., 50., 51.,  ..., 96., 97., 98.]])
padding:  1
after conv
tensor([[  2.4100,   6.3100,   9.9100,  13.5100,  17.1100,  20.7100,  24.3100,
          27.9100,  31.5100,  35.1100,  38.7100,  42.3100,  45.9100,  49.5100,
          53.1100,  56.7100,  60.3100,  63.9100,  67.5100,  71.1100,  74.7100,
          78.3100,  81.9100,  85.5100,  89.1100],
        [  6.3100,  13.5100,  18.9100,  24.3100,  29.7100,  35.1100,  40.5100,
          45.9100,  51.3100,  56.7100,  62.1100,  67.5100,  72.9100,  78.3100,
          83.7100,  89.1100,  94.5100,  99.9100, 105.3100, 110.7100, 116.1100,
         121.5100, 126.9100, 132.3100, 137.7100],
        [  9.9100,  18.9100,  24.3100,  29.7100,  35.1100,  40.5100,  45.9100,
          51.310